In [1]:
#Runtime -> Change runtime type -> Hardware accelerator=GPU
!nvidia-smi

Wed Jan 18 06:31:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Gerekli Kütüphaneleri Ekleme

In [1]:
import tensorflow
import keras
from keras import datasets, layers, models
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

print("Tensorflow version: ", tensorflow.__version__)
print("Keras version: ", keras.__version__)
print("Numpy version: ", np.__version__)
print("matplotlib version: ", matplotlib.__version__)

Tensorflow version:  2.9.2
Keras version:  2.9.0
Numpy version:  1.21.6
matplotlib version:  3.2.2


## Veri Kümesini İndirme
### Eğitim ve Test kümelerine ayırma

In [2]:
#Download datasets
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()

print(X_train.shape)
print(X_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


Cifar10 veri kümesi içerisinde 50bin adet eğitim örneği, 10bin adet test örneği mevcut.

## Önişlemler

In [3]:
# 2 boyutlu diziyi tek boyutlu hale dönüştürüyoruz
y_test = y_test.reshape(
    -1,
)

# sınıflarımız
image_classes = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

# normalizasyon | doğru sonuç almak için renk pixeline bölüyoruz
X_train = X_train / 255
X_test = X_test / 255

# MODEL

In [4]:
deep_learning_model = models.Sequential(
    [
        layers.Conv2D(  # bu görsellerden belirli özellikleri yakalamaya çalışır.
            filters=32, kernel_size=(3, 3), activation="relu", input_shape=(32, 32, 3)
        ),
        layers.MaxPooling2D(
            (2, 2)
        ),  # en büyük pixel değerini alır en belirgin özellikleri ortaya çıkarır işlem hızını etiler
        layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),  # CNN ve ANN i otomatik olarak bağlıyoruz
        layers.Dense(
            64, activation="relu"
        ),  # relu negatif değerler için 0 pozitif x değerler için x i alır. böylece ağ daha hızlı eğitilir
        layers.Dense(
            10, activation="softmax"
        ),
        # softmax sınıflandırma problemlerinde kullanılır her girdinin bir sınıfa ait olmasını gösteren 0 veya 1 degerini alır
    ]
)


# çoklu sınıfa sahip olduğu için loss olarak "sparse_categorical_crossentropy" seçtik.
# accuracy skorumuzu ölçerek model başarımını değerlendireceğiz.
# optimizer yöntemlerinden birisi olan "adam"ı seçtik.
deep_learning_model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

## Model Eğitimi - Training

In [5]:
history = deep_learning_model.fit(X_train, y_train, epochs=20)

Epoch 1/20
1563/1563 [==============================] - 8s 4ms/step - loss: 1.4532 - accuracy: 0.4768
Epoch 2/20
1563/1563 [==============================] - 6s 4ms/step - loss: 1.1042 - accuracy: 0.6117
Epoch 3/20
1563/1563 [==============================] - 6s 4ms/step - loss: 0.9753 - accuracy: 0.6592
Epoch 4/20
1563/1563 [==============================] - 6s 4ms/step - loss: 0.8914 - accuracy: 0.6898
Epoch 5/20
1563/1563 [==============================] - 6s 4ms/step - loss: 0.8224 - accuracy: 0.7147
Epoch 6/20
1563/1563 [==============================] - 6s 4ms/step - loss: 0.7655 - accuracy: 0.7321
Epoch 7/20
1563/1563 [==============================] - 6s 4ms/step - loss: 0.7212 - accuracy: 0.7490
Epoch 8/20
1563/1563 [==============================] - 6s 4ms/step - loss: 0.6793 - accuracy: 0.7635
Epoch 9/20
1563/1563 [==============================] - 6s 4ms/step - loss: 0.6321 - accuracy: 0.7797
Epoch 10/20
1563/1563 [==============================] - 6s 4ms/step - loss: 0.592

## Model Başarım Değerlendirme - Evaluate

In [7]:
deep_learning_model.evaluate(X_test, y_test)  # model başarısını ölçüyoruz

313/313 [==============================] - 1s 3ms/step - loss: 1.3364 - accuracy: 0.6861


[1.3364291191101074, 0.6861000061035156]

## TEST veri kümesiyle tahmin sonuçlarını karşılaştırma

In [9]:
prediction = deep_learning_model.predict(X_test)

predictions_classes = [np.argmax(element) for element in prediction]

for i in range(20):
    test_result = image_classes[y_test[i]]
    prediction_result = image_classes[predictions_classes[i]]
    if str(test_result) == str(prediction_result):
        print("test verisi: " + str(test_result) + " Tahmin: " + str(prediction_result) + " || DOĞRU")
    else:
        print("test verisi: " + str(test_result) + " Tahmin: " + str(prediction_result) + " || YANLIŞ")


313/313 [==============================] - 1s 4ms/step
test verisi: cat Tahmin: ship || YANLIŞ
test verisi: ship Tahmin: ship || DOĞRU
test verisi: ship Tahmin: ship || DOĞRU
test verisi: airplane Tahmin: airplane || DOĞRU
test verisi: frog Tahmin: frog || DOĞRU
test verisi: frog Tahmin: frog || DOĞRU
test verisi: automobile Tahmin: automobile || DOĞRU
test verisi: frog Tahmin: bird || YANLIŞ
test verisi: cat Tahmin: cat || DOĞRU
test verisi: automobile Tahmin: truck || YANLIŞ
test verisi: airplane Tahmin: airplane || DOĞRU
test verisi: truck Tahmin: truck || DOĞRU
test verisi: dog Tahmin: cat || YANLIŞ
test verisi: horse Tahmin: horse || DOĞRU
test verisi: truck Tahmin: truck || DOĞRU
test verisi: ship Tahmin: ship || DOĞRU
test verisi: dog Tahmin: dog || DOĞRU
test verisi: horse Tahmin: dog || YANLIŞ
test verisi: ship Tahmin: ship || DOĞRU
test verisi: frog Tahmin: frog || DOĞRU
